In [103]:
import numpy as np
import sympy as sp

In [104]:
AU = 149597870 #km

def MOID(kep1, kep2):
    """
    Calculate the Minimum Orbit Intersection Distance (MOID) between two orbits
    Inputs:
        kep1: array of orbital elements [a, e, i, Ω, ω] for the first orbit
        kep2: array of orbital elements [a, e, i, Ω, ω] for the second orbit
    Outputs:
        MOID: Minimum Orbit Intersection Distance between the two orbits
    """

    a1, e1, i1, argp1, raan1 = kep1
    a2, e2, i2, argp2, raan2 = kep2

    # Step size constants
    scanning_step = 0.12 # initial scanning step size (rad)
    ini_ini_tuning_step = 0.07 # initial step size of first tuning (rad)
    fin_ini_tuning_step = 1e-10 # final step size of first tuning (rad)

    # ----------------------------------------------------------------------------------
    # Step 0: transition matrix rotates to frame with first orbit in the reference plane
    # ----------------------------------------------------------------------------------
    # First orbit rotation matrix
    R11 = np.cos(raan1) * np.cos(argp1) - np.sin(raan1) * np.cos(i1) * np.sin(argp1)
    R12 = np.sin(raan1) * np.cos(argp1) + np.cos(raan1) * np.cos(i1) * np.sin(argp1)
    R13 = np.sin(i1) * np.sin(argp1)
    R21 = -np.cos(raan1) * np.sin(argp1) - np.sin(raan1) * np.cos(i1) * np.cos(argp1)
    R22 = -np.sin(raan1) * np.sin(argp1) + np.cos(raan1) * np.cos(i1) * np.cos(argp1)
    R23 = np.sin(i1) * np.cos(argp1)
    R31 = np.sin(i1) * np.sin(raan1)
    R32 = -np.sin(i1) * np.cos(raan1)
    R33 = np.cos(i1)

    # Second orbit rotation matrix
    x1 = np.cos(raan2) * np.cos(argp2) - np.sin(raan2) * np.cos(i2) * np.sin(argp2)
    x2 = np.sin(raan2) * np.cos(argp2) + np.cos(raan2) * np.cos(i2) * np.sin(argp2)
    x3 = np.sin(i2) * np.sin(argp2)
    y1 = -np.cos(raan2) * np.sin(argp2) - np.sin(raan2) * np.cos(i2) * np.cos(argp2)
    y2 = -np.sin(raan2) * np.sin(argp2) + np.cos(raan2) * np.cos(i2) * np.cos(argp2)
    y3 = np.sin(i2) * np.cos(argp2)
    z1 = np.sin(i2) * np.sin(raan2)
    z2 = -np.sin(i2) * np.cos(raan2)
    z3 = np.cos(i2)
    
    # Values needed to calculate new i, raan, w
    z1n = R11 * z1 + R12 * z2 + R13 * z3
    z2n = R21 * z1 + R22 * z2 + R23 * z3
    z3n = R31 * z1 + R32 * z2 + R33 * z3
    y3n = R31 * y1 + R32 * y2 + R33 * y3
    x3n = R31 * x1 + R32 * x2 + R33 * x3

    # New inclination, raan, and arg peri
    i2 = np.arctan2(np.sqrt(z1n * z1n + z2n * z2n), z3n)
    raan2 = -np.arctan2(z1n, -z2n)
    argp2 = -np.arctan2(x3n, y3n)
    i1, raan1, argp1 = 0, 0, 0

    # distance function to be used later
    def dist(L, TA):
        """
        Finds distance between orbit one and two give the first's longitude and the second's true anomaly
        Inputs:
            L: longitude of first orbit (rad)
            TA: true anomaly of second orbit (rad)
        Output:
            Distance between objects
        """

        r1 = a1 * (1 - e1**2) / (1 + e1 * np.cos(L))
        r2 = a2 * (1 - e2**2) / (1 + e2 * np.cos(TA))

        x1 = r1 * np.cos(L)
        y1 = r1 * np.sin(L)
        z1 = 0

        x2 = r2 * (np.cos(raan2) * np.cos(argp2 + TA) - np.sin(raan2) * np.sin(argp2 + TA) * np.cos(i2))
        y2 = r2 * (np.sin(raan2) * np.cos(argp2 + TA) + np.cos(raan2) * np.sin(argp2 + TA) * np.cos(i2))
        z2 = r2 * (np.sin(argp2 + TA) * np.sin(i2))

        D = np.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)

        return D
    
    # ---------------------------------------
    # Step 1: scanning along meridional plane
    # ---------------------------------------
    ta2 = 0
    d_min_list = [] # list of distances
    ta_min_list = [] # list of true anomalies of second orbit
    l_min_list = [] # list of longitudes of first orbit
    
    d0_triple = np.array([0, 0, 0]) # list of distances to decide on local min
    ta_triple = np.array([0, 0, 0])
    l_triple = np.array([0, 0, 0])
    def update_triple(val, list):
        list[2] = list[1]
        list[1] = list[0]
        list[0] = val

    while ta2 < 2 * np.pi + 3 * scanning_step:
        # calculate position of second orbit
        r20 = a2 * (1 - e2**2) / (1 + e2 * np.cos(ta2))
        x20 = r20 * (np.cos(raan2) * np.cos(argp2 + ta2) - np.sin(raan2) * np.sin(argp2 + ta2) * np.cos(i2))
        y20 = r20 * (np.sin(raan2) * np.cos(argp2 + ta2) + np.cos(raan2) * np.sin(argp2 + ta2) * np.cos(i2))
        l0 = np.atan2(y20, x20)

        # calculate distance between orbits
        d0 = dist(l0, ta2)

        # check for local minimum
        update_triple(d0, d0_triple)
        update_triple(ta2, ta_triple)
        update_triple(l0, l_triple)
        if d0_triple[1] < d0_triple[0] and d0_triple[1] < d0_triple[2]:
            d_min_list.append(d0_triple[1])
            ta_min_list.append(ta_triple[1])
            l_min_list.append(l_triple[1])
        
        # move scan forward
        ta2 = ta2 + scanning_step

    d_min_list = np.array(d_min_list)
    ta_min_list = np.array(ta_min_list)
    l_min_list = np.array(l_min_list)

    # Water procedure: distributes points evenly if only one min exists
    if ta_min_list.size == 1:
        ta_min_list = np.array([np.pi / 8, 5 * np.pi / 8, 9 * np.pi / 8, 13 * np.pi / 8])
        for i in range(ta_min_list.size):
            r20 = a2 * (1 - e2**2) / (1 + e2 * np.cos(ta_min_list[i]))
            x20 = r20 * (np.cos(raan2) * np.cos(argp2 + ta_min_list[i]) - np.sin(raan2) * np.sin(argp2 + ta_min_list[i]) * np.cos(i2))
            y20 = r20 * (np.sin(raan2) * np.cos(argp2 + ta_min_list[i]) + np.cos(raan2) * np.sin(argp2 + ta_min_list[i]) * np.cos(i2))
            l0 = np.atan2(y20, x20)

            # calculate distance between orbits
            d0 = dist(l0, ta_min_list[i])
            l_min_list[i] = l0
            d_min_list[i] = d0

    # --------------------
    # Step 2: tuning
    # --------------------
    for i in range(ta_min_list.size):
        tuning_step = ini_ini_tuning_step
        TA = ta_min_list[i]
        L = l_min_list[i]
        D = d_min_list[i]
        while tuning_step > fin_ini_tuning_step:
            not_finished = True
            D_arr = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0])
            while not_finished:
                L_l = (L - tuning_step)
                L_r = (L + tuning_step)
                TA_l = (TA - tuning_step)
                TA_r = (TA + tuning_step)

                TA_arr = np.array([TA, TA_l, TA_r, TA, TA_l, TA_r, TA, TA_l, TA_r])
                L_arr = np.array([L, L, L, L_l, L_l, L_l, L_r, L_r, L_r])
                D_arr[0] = dist(L, TA)
                D_arr[1] = dist(L, TA_l)
                D_arr[2] = dist(L, TA_r)
                D_arr[3] = dist(L_l, TA)
                D_arr[4] = dist(L_l, TA_l)
                D_arr[5] = dist(L_l, TA_r)
                D_arr[6] = dist(L_r, TA)
                D_arr[7] = dist(L_r, TA_l)
                D_arr[8] = dist(L_r, TA_r)
                
                min_idx = np.argmin(D_arr)
                # potential print functions to see each step of tuning
                # print(f"Tuning: TA = {TA}, L = {L}, D = {D / 1.496e+8}, min_idx = {min_idx}")
                # print(f"D_arr: {D_arr / 1.496e+8}")
                TA = TA_arr[min_idx]
                L = L_arr[min_idx]
                D = D_arr[min_idx]

                if min_idx == 0: 
                    not_finished = False
            ta_min_list[i] = TA
            l_min_list[i] = L
            d_min_list[i] = D 
            tuning_step = tuning_step * 0.2

    for i in range(ta_min_list.size):
        print(f"    Min {i}: TA = {ta_min_list[i]} rad, L = {ta_min_list[i]} rad, D = {d_min_list[i] / AU} AU")
    moid_idx = np.argmin(d_min_list)
    print(f"    MOID: TA = {ta_min_list[moid_idx]} rad, L = {l_min_list[moid_idx]} rad, D = {d_min_list[moid_idx] / AU} AU")

In [105]:
# a1, e1, i1, argp1, raan1 = kep1

earth_kep = [
    1.4765067E+08,
    9.1669995E-03,
    np.radians(4.2422693E-03),
    np.radians(6.64375167E+01),
    np.radians(1.4760836E+01)
]

apophis_kep = [
    1.3793939E+08,
    1.9097084E-01,
    np.radians(3.3356539E+00),
    np.radians(1.2919949E+02),
    np.radians(2.0381969E+02)
]

yr4_kep = [
    3.7680703E+08,
    6.6164147E-01,
    np.radians(3.4001497E+00),
    np.radians(1.3429905E+02),
    np.radians(2.7147904E+02)
]

atlas_kep = [
    -3.9552667E+07,
    6.1469268E+00,
    np.radians(1.7512507E+02),
    np.radians(1.2817255E+02),
    np.radians(3.2228906E+02)
]

In [106]:
print(f"Earth-Apophis")
MOID(earth_kep, apophis_kep)

print(f"Earth-YR4")
MOID(earth_kep, yr4_kep)

print(f"Apophis-YR4")
MOID(apophis_kep, yr4_kep)

print(f"Earth-Atlas")
MOID(earth_kep, atlas_kep)

Earth-Apophis
    Min 0: TA = 2 rad, L = 2 rad, D = 0.00566783470914392 AU
    Min 1: TA = 4 rad, L = 4 rad, D = 0.05220350396700167 AU
    MOID: TA = 2 rad, L = -2 rad, D = 0.00566783470914392 AU
Earth-YR4
    Min 0: TA = 0 rad, L = 0 rad, D = 0.05833984802056339 AU
    Min 1: TA = 5 rad, L = 5 rad, D = 0.001767759126516975 AU
    MOID: TA = 5 rad, L = 0 rad, D = 0.001767759126516975 AU
Apophis-YR4
    Min 0: TA = 5 rad, L = 5 rad, D = 0.050339333039969084 AU
    Min 1: TA = 6 rad, L = 6 rad, D = 0.055246180978378905 AU
    MOID: TA = 5 rad, L = -2 rad, D = 0.050339333039969084 AU
Earth-Atlas
    Min 0: TA = 3 rad, L = 3 rad, D = 0.9037985768112875 AU
    Min 1: TA = 6 rad, L = 6 rad, D = 0.37841542797367367 AU
    MOID: TA = 6 rad, L = -1 rad, D = 0.37841542797367367 AU
